In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import os
import re

In [1]:
reviews_train = []
for line in open('newFile.csv', 'r'):
    
    reviews_train.append(line.strip())
    
reviews_test = []
for line in open('newFile.csv', 'r'):
    
    reviews_test.append(line.strip())

In [2]:
reviews_train[5]

'"b\'The production value of Light Emitting Diode (LED) Products has gone up from INR 7,134 crore in 2016-17 to INR 9,63\\xe2\\x80\\xa6 https://t.co/N61TnioSvN\'"'

In [3]:
import re


#REPLACE_NO_SPACE = re.compile("virginamerica")
REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\@)|(\d+)|(\virginamerica)")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
NO_SPACE = ""
SPACE = " "

def preprocess_reviews(reviews):
    
    reviews = [REPLACE_NO_SPACE.sub(NO_SPACE, line.lower()) for line in reviews]
    reviews = [REPLACE_WITH_SPACE.sub(SPACE, line) for line in reviews]
    
    return reviews

reviews_train_clean = preprocess_reviews(reviews_train)
reviews_test_clean = preprocess_reviews(reviews_test)

In [4]:
reviews_train_clean[5]

'bthe production value of light emitting diode led products has gone up from inr  crore in   to inr \\xe\\x\\xa https  tco ntniosvn'

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(binary=True)
cv.fit(reviews_train_clean)
X = cv.transform(reviews_train_clean)
X_test = cv.transform(reviews_test_clean)

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

target = [1 if i < 100 else 0 for i in range(1001)]

X_train, X_val, y_train, y_val = train_test_split(
    X, target, train_size = 0.75
)

for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    lr = LogisticRegression(C=c)
    lr.fit(X_train, y_train)
    print ("Accuracy for C=%s: %s" 
           % (c, accuracy_score(y_val, lr.predict(X_val))))
    
#     Accuracy for C=0.01: 0.87616
#     Accuracy for C=0.05: 0.88736
#     Accuracy for C=0.25: 0.88768
#     Accuracy for C=0.5: 0.88336
#     Accuracy for C=1: 0.8784

Accuracy for C=0.01: 0.8924302788844621
Accuracy for C=0.05: 0.896414342629482
Accuracy for C=0.25: 0.900398406374502
Accuracy for C=0.5: 0.896414342629482
Accuracy for C=1: 0.900398406374502


/home/poochi/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [9]:
final_model = LogisticRegression(C=0.054)
final_model.fit(X, target)
print ("Final Accuracy: %s" 
       % accuracy_score(target, final_model.predict(X_test)))
# Final Accuracy: 0.88048

Final Accuracy: 0.9080919080919081


In [10]:
feature_to_coef = {
    word: coef for word, coef in zip(
        cv.get_feature_names(), final_model.coef_[0]
    )
}
for best_positive in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1], 
    reverse=True)[:5]:
    print (best_positive)
    
print("___________________________________")
    
for best_negative in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1])[:5]:
    print (best_negative)
    
#     ('worst', -1.367978497228895)
#     ('waste', -1.1684451288279047)
#     ('awful', -1.0277001734353677)
#     ('poorly', -0.8748317895742782)
#     ('boring', -0.8587249740682945)

('budget', 0.4721068014248552)
('pib_india', 0.2867271689459507)
('cr', 0.2493124916866458)
('fm', 0.23070223264011136)
('mobile', 0.21570704480085295)
___________________________________
('brt', -0.6359916126393377)
('xe', -0.3477459253325596)
('at', -0.3437535085250574)
('by', -0.3430088136128233)
('xa', -0.3292469597535217)
